# Lab 6: Cross Validation and Gradient Descent
## by Tiffany Nguyen
The purpose of this lab is to conduct cross validtion for selecting an optimal lambda for ridge regression and implement gradient descent from scratch using just the NumPy library
### Part 1: Read Data

In [1]:
import numpy as np

In [2]:
# Read all training data
trainDataX = np.empty((0, 95))
trainDataY = np.empty((0))

#read train data
with open("crime-train.txt", "r") as filestream:
  next(filestream) #skip header line
  for line in filestream:
    currentline = line.strip().split("\t")
    trainDataX = np.append(trainDataX, [np.array(currentline[1:], dtype=float)], axis=0)
    trainDataY = np.append(trainDataY, float(currentline[0]))

# add dummy feature to train data (column of ones for bias) 
dummyCol = np.ones((trainDataX.shape[0], 1))
trainDataX = np.concatenate((trainDataX, dummyCol), axis=1)

# print output
print("trainDataX shape:", trainDataX.shape)
print("trainDataY shape:", trainDataY.shape)
# print("trainDataX: \n", trainDataX)
# print("trainDataY: \n", trainDataY)

trainDataX shape: (1595, 96)
trainDataY shape: (1595,)


In [3]:
# Read all testing data
testDataX = np.empty((0, 95))
testDataY = np.empty((0))

#read train data
with open("crime-test.txt", "r") as filestream:
  next(filestream) #skip header line
  for line in filestream:
    currentline = line.strip().split("\t")
    testDataX = np.append(testDataX, [np.array(currentline[1:], dtype=float)], axis=0)
    testDataY = np.append(testDataY, float(currentline[0]))

# add dummy feature to train data (column of ones for bias) 
dummyCol = np.ones((testDataX.shape[0], 1))
testDataX = np.concatenate((testDataX, dummyCol), axis=1)

# print output
print("testDataX shape:", testDataX.shape)
print("testDataY shape:", testDataY.shape)
# print("testDataX: \n", testDataX)
# print("testDataY: \n", testDataY)

testDataX shape: (399, 96)
testDataY shape: (399,)


### Part 2: K-fold Cross Validation
**Selected Lambda**: My selected Lambda is 25, and its RMSE is 0.13591585919851965. I selected 25 becuase, after performing k-fold cross validation and getting the average RMSE of all folds for each lambda, the average RMSE was smallest when lambda was 25.

In [4]:
# Divide data into k sets
k = 5
numPerSet = int(trainDataX.shape[0] / k)
print("numPerSet:", numPerSet)

fold1X = trainDataX[0:numPerSet]
fold2X = trainDataX[numPerSet:2*numPerSet]
fold3X = trainDataX[2*numPerSet:3*numPerSet]
fold4X = trainDataX[3*numPerSet:4*numPerSet]
fold5X = trainDataX[4*numPerSet:5*numPerSet]
allFolds = np.array([fold1X, fold2X, fold3X, fold4X, fold5X])
# print(allFolds[0].shape)

fold1Y = trainDataY[0:numPerSet]
fold2Y = trainDataY[numPerSet:2*numPerSet]
fold3Y = trainDataY[2*numPerSet:3*numPerSet]
fold4Y = trainDataY[3*numPerSet:4*numPerSet]
fold5Y = trainDataY[4*numPerSet:5*numPerSet]
allFoldsY = np.array([fold1Y, fold2Y, fold3Y, fold4Y, fold5Y])


numPerSet: 319


In [6]:
# Create list to store lambda and RMSE values
lambdaVal = 400 #initial lambda value
allLambda = np.empty((10))
for i in range(10):
  allLambda[i] = int(lambdaVal)
  lambdaVal /= 2
print("All Lambda Values:", allLambda)
allRMSE = np.empty((10))

All Lambda Values: [400. 200. 100.  50.  25.  12.   6.   3.   1.   0.]


In [7]:
# Perform k-fold cross test validation
for i in range(len(allLambda)):
  curRMSE = np.empty((k)) # all RMSE associated with current lambda value for each k fold "set"
  
  # Calculate RMSE all fold combinations
  for curK in range(k):
    # get train and validation data from folds
    initialIndex = 0 if curK != 0 else 1
    curTrainX = allFolds[initialIndex]
    curTrainY = allFoldsY[initialIndex]
    for curFoldIndex in range(len(allFolds)):
        if(curFoldIndex != curK and curFoldIndex != initialIndex):
          curTrainX = np.concatenate((curTrainX, allFolds[curFoldIndex]), axis=0)
          curTrainY = np.concatenate((curTrainY, allFoldsY[curFoldIndex]), axis=0)
    curValidX = allFolds[curK]
    curValidY = allFoldsY[curK]
    # print(curValid.shape) # should be (319, 96)
    # print(curTrainX.shape) # should be (1276, 96)

    # calculate ridge regression weight
    xtx = np.dot(np.transpose(curTrainX), curTrainX)
    lambdaIdentity = allLambda[i] * np.identity(curTrainX.shape[1])
    xtxLambdaInv = np.linalg.inv(xtx + lambdaIdentity)
    xtxLambdaInvXt = np.dot(xtxLambdaInv, np.transpose(curTrainX))
    ridgeRegressionWeights = np.dot(xtxLambdaInvXt, curTrainY)

    # get ridge regression prediction using validation data
    ridgePredValid = np.dot(ridgeRegressionWeights, np.transpose(curValidX))

    # evaluate RMSE for current ridge regression
    curRMSE[curK] = np.sqrt(np.sum(np.square(ridgePredValid - curValidY))/len(ridgePredValid))

  # get average RMSE for the given fold
  allRMSE[i] = np.average(curRMSE)

# find optimal lambda using min RMSE
print("All RMSE:", allRMSE)
print("Min RMSE:", np.min(allRMSE))
optimalLambda = allLambda[np.argmin(allRMSE)]
print("Optimal Lambda", optimalLambda)



All RMSE: [0.14951278 0.14069402 0.13727677 0.13615594 0.13591586 0.13603416
 0.1362846  0.13658964 0.13707909 0.13777474]
Min RMSE: 0.13591585919851965
Optimal Lambda 25.0


### Part 3: Gradient Descent
Equation to update the weights:
$$ w_{t+1} = w_t + \alpha \Delta L(w_t)$$
For Linear Regression, this becomes
$$ w_{t+1} = w_t + \alpha  x^T(y-xw^T)$$

In [8]:
# declare variables
initialWeight = np.random.normal()
alpha = 0.00001
convergeMax = 10**-5
print("initial weight:", initialWeight)

initial weight: -1.9712730245587566


In [10]:
# Perform gradient descent with linear regression
def problem2(samplesX, samplesY):
  np.random.seed(0)
  curWeight = np.random.normal(0, 1, samplesX[0].shape)
  prevWeight = -1 * np.ones(samplesX[0].shape)

  print(max(curWeight))

  while(max(np.subtract(curWeight, prevWeight)) > convergeMax):
    prevWeight = curWeight
    update = alpha * (np.dot(samplesX.T, np.subtract(samplesY, np.dot(samplesX, prevWeight.T))))
    curWeight = prevWeight + update

    # get and print current RMSE
    predTrain = np.dot(curWeight, np.transpose(samplesX))
    curRMSE =  np.sqrt(np.sum(np.square(predTrain - samplesY))/len(predTrain))
    print("curRMSE", curRMSE, end="\r")
    
  return(curWeight)

gradDescWeights = problem2(trainDataX, trainDataY)
print("\nweights", gradDescWeights)

2.2697546239876076
curRMSE 0.13688241913867798
weights [ 1.74107285e-01  2.48577605e-02 -2.25011068e-02 -6.70545547e-02
  1.41280495e-02 -1.82205037e-03 -2.03715959e-01  3.92210862e-02
 -3.94820146e-02 -4.40020252e-02  6.20517485e-03 -4.20224488e-02
 -2.75008292e-03 -3.50988836e-03 -7.79593411e-03  5.35526601e-03
  1.55149196e-01 -1.47382538e-01 -1.77154372e-03 -2.68049761e-03
  3.64686951e-03  1.14245320e-02  1.93869076e-02 -5.35700751e-02
  5.07096933e-03 -2.34679453e-02  1.97118592e-02  2.60153974e-03
  3.91621197e-02 -1.31035942e-02 -1.20558416e-03  2.67522038e-02
  7.90170457e-03 -4.35640562e-01  6.12077382e-02 -5.75971870e-01
  9.77420097e-01  2.75798483e-02  1.22277120e-01 -2.10792588e-01
 -1.11361923e-02 -1.11646924e-03  3.05582499e-03 -2.20938416e-02
 -3.53119880e-03  5.24184760e-02 -2.36766026e-02  1.01922078e-03
 -1.80181373e-02  5.30366972e-02 -3.59065175e-02 -2.00009456e-02
  1.45576588e-01 -4.04243753e-01  3.22160090e-01 -2.80986941e-02
 -8.81770838e-02  1.46863809e-01 -1

In [11]:
# Evaluate using training dataset
predTrain = np.dot(gradDescWeights, np.transpose(trainDataX))
trainRMSE =  np.sqrt(np.sum(np.square(predTrain - trainDataY))/len(predTrain))
print("Train RMSE", trainRMSE)

# Evaluate using testing dataset
predTest = np.dot(gradDescWeights, np.transpose(testDataX))
testRMSE =  np.sqrt(np.sum(np.square(predTest - testDataY))/len(predTest))
print("Test RMSE", testRMSE)

Train RMSE 0.1368824191386779
Test RMSE 0.1551456595144984


### Part 4: Ridge Regression with Cross Validation and Graident Descent
Weight Update Equation for Ridge Regression:
$$w_{t+1} = w_t + \alpha[X^T(y-Xw^T)-\lambda w^T]$$

In [ ]:
def problem3():
  k = 5
  # Perform k-fold cross test validation
  for i in range(len(allLambda)):
    print("Lambda", allLambda[i])
    curK_RMSE = np.empty((k)) # all RMSE associated with current lambda value for each k fold "set"
    # Calculate RMSE all fold combinations
    for curK in range(k):
      print("current k", curK)
      # get train and validation data from folds
      initialIndex = 0 if curK != 0 else 1
      curTrainX = allFolds[initialIndex]
      curTrainY = allFoldsY[initialIndex]
      for curFoldIndex in range(len(allFolds)):
          if(curFoldIndex != curK and curFoldIndex != initialIndex):
            curTrainX = np.concatenate((curTrainX, allFolds[curFoldIndex]), axis=0)
            curTrainY = np.concatenate((curTrainY, allFoldsY[curFoldIndex]), axis=0)
      curValidX = allFolds[curK]
      curValidY = allFoldsY[curK]

      # calculate ridge regression weight using graident descent
      curWeight = np.random.normal(0, 1, curTrainX[0].shape)
      prevWeight = -1 * np.ones(curTrainX[0].shape)
      while(max(np.subtract(curWeight, prevWeight)) > convergeMax):
        prevWeight = curWeight
        update = alpha * np.subtract(np.dot(curTrainX.T, np.subtract(curTrainY, np.dot(curTrainX, prevWeight.T))), np.dot(allLambda[i], prevWeight.T))
        curWeight = prevWeight + update

        # get and print current RMSE
        predTrain = np.dot(curWeight, np.transpose(curTrainX))
        curRMSE =  np.sqrt(np.sum(np.square(predTrain - curTrainY))/len(predTrain))
        print("curRMSE", curRMSE, end="\r")

      # get ridge regression prediction using validation data
      ridgePredValid = np.dot(curWeight, np.transpose(curValidX))

      # evaluate RMSE for current ridge regression
      curK_RMSE[int(curK)] = np.sqrt(np.sum(np.square(ridgePredValid - curValidY))/len(ridgePredValid))
      print("\n") #get rid of \r end

    # get average RMSE for the given fold
    allRMSE[i] = np.average(curK_RMSE)

  # find optimal lambda using min RMSE
  print("All RMSE:", allRMSE)
  print("Min RMSE:", np.min(allRMSE))
  optimalLambda = allLambda[np.argmin(allRMSE)]
  print("Optimal Lambda", optimalLambda)
  return(optimalLambda)
optimalLambda = problem3()

Lambda 400.0
